### Importing relevant libraries for data imports and webscraping

In [718]:
import os
import pandas as pd
import sqlite3

#Libraries for dateime
from datetime import datetime
from datetime import timedelta

#Libraries for webscraping
import requests
import json
from bs4 import BeautifulSoup

### BIXI Data

In [719]:
folder = 'BixiRentals2018'  # instert folder name containing all bixi transactions for the year

def get_bixi_data(folder):
    """
    Returns dataframe combining all .csv files contained in supplied folder name into one dataframe
    sorted by date of transaction
    """
    
    files = []
    for file in os.listdir(folder):
        if file.endswith('.csv'):
            files.append(pd.read_csv(f'{folder}/{file}'))

    df = pd.concat(files, axis=0, ignore_index=True)
    df = df.sort_values(by='start_date')
    
    # assign name to df to later save into a SQL table
    df.name = 'bixi'
    
    return df

In [720]:
bixi = get_bixi_data(folder)

### Webscraping

#### Part 1 : Weather data from wunderground.com - April-November 2018 inclusive as BIXI is available only part of the year in Montreal.

Source: https://www.wunderground.com/history/daily/ca/montreal/CYUL/

In [762]:
lat = '45.445'
long = '-73.751'
key = '6532d6454b8aa370768e63d6ba5a832e'

def get_weather_month(lat,long,key,start,end):
    """
    Returns dataframe with weather scraped from api/weather.com for the latitude & longitude mentioned
    and start to end date. There is a maximum of 31 days per request
    """
    
    url = f'https://api.weather.com/v1/geocode/{lat}/{long}/observations/historical.json?apiKey={key}&startDate={start}&endDate={end}&units=e'
    results = requests.get(url).json()['observations']
    df = pd.DataFrame([result for result in results])
    
    # Columns to keep
    keep_columns = ['valid_time_gmt','day_ind','feels_like','heat_index','precip_hrly',
                    'rh','temp','uv_desc','uv_index','wc','wspd','wx_phrase']
    
    # Creates df with columns to keep
    df = df[keep_columns]
    
    # Converting Greenwich meantime timestamp into date time
    df['valid_time_gmt'] = df['valid_time_gmt'].apply(lambda x: datetime.fromtimestamp(x).strftime('%Y-%m-%d %H:%M'))
    df['valid_time_gmt'] = pd.to_datetime(df['valid_time_gmt'])
    
    # Extract date & time from timestamp
    df['date'] = df['valid_time_gmt'].apply(lambda x: datetime.date(x))
    df['time'] = df['valid_time_gmt'].apply(lambda x: datetime.time(x))
    df.drop(['valid_time_gmt'], axis = 1, inplace = True)
    
    # Renaming for better understanding
    # rh stands for relative humidity and wx_phrase stands for weather condition
    df.rename(columns={'rh':'humidity','wx_phrase':'condition','wspd':'wind_speed'}, inplace=True)

    return df

In [722]:
def get_all_weather(lat, long, key, year):
    """
    Returns dataframe combining weather information for supplied latitude and longitude for the months
    of April-November inclusive for the specified year
    """
    
    #Calling get_weather function for needed months
    apr = get_weather(lat,long,key,f'{year}0401',f'{year}0430')
    may = get_weather(lat,long,key,f'{year}0501',f'{year}0531')
    jun = get_weather(lat,long,key,f'{year}0601',f'{year}0630')
    jul = get_weather(lat,long,key,f'{year}0701',f'{year}0731')
    aug = get_weather(lat,long,key,f'{year}0801',f'{year}0831')
    sep = get_weather(lat,long,key,f'{year}0901',f'{year}0930')
    octo = get_weather(lat,long,key,f'{year}1001',f'{year}1030')
    nov = get_weather(lat,long,key,f'{year}1101',f'{year}1130')
    
    #Bixi is available for April to November inclusive
    months = [apr,may,jun,jul,aug,sep,octo,nov]
    
    df = pd.concat(months, axis=0, ignore_index=True)
    
    # assign name to df to later save into a SQL table
    df.name = 'weather'
    
    return df

In [723]:
weather = get_all_weather(lat,long,key,year)

#### Part 2: Scraping public holidays

Source: https://www.officeholidays.com/countries/canada/quebec/2018.php

In [724]:
year = 2018

def get_holidays(year):
    """Returns dataframe with scraped holidays for the province of Quebec for the supplied year"""

    url = f'https://www.officeholidays.com/countries/canada/quebec/{year}.php'
    html = requests.get(url)
    
    # Pass the page contents to beautiful soup for parsing
    soup = BeautifulSoup(html.content, 'html.parser') 
        
    # Get holiday dates
    holidays = ["".join(i.contents) for i in soup.find_all('time')]
    
    # Get holiday description
    holiday_desc = [i.contents for i in soup.find_all('a')][-22:-11]
    holiday_desc = [val.strip('  ') for holiday in holiday_desc for val in holiday]
    
    # Create dataframe with Holiday & Description
    df = pd.DataFrame(list(zip(holidays, desc_holidays)), columns= ['holidays','holiday_desc'])
    
    # assign name to df to later save into a SQL table
    df.name = 'holiday'
    
    return df

In [725]:
holidays = get_holidays(2018)

### Saving results to SQL database

In [751]:
dfs = [bixi, weather, holidays]

In [749]:
def save_to_sql(dfs, db_name):
    """
    Creates a SQL database with the specified name and saves dataframes
    listed in dfs database
    """
    
    # Create database

    connex = sqlite3.connect(f'{db_name}.db')  
    cur = connex.cursor()

    # Create SQL tables in database
    for df in dfs:
        df.to_sql(name = df.name, con=connex, if_exists="replace", index=False)  
    
    # Close 
    connex.close()

In [752]:
save_to_sql(dfs, 'bixi_data')